# Hloubkové senzory - měření objektů
Cvičení je zaměřené na práci s [hloubkovými mapami](https://en.wikipedia.org/wiki/Depth_map) vzniklých použitím tzv. hloubkových senzorů. V laboratoři jsou k dispozici hloubkové senzory založené na několika principech zisku hloubkových map. Jedná se o kamery typu Kinect, RealSense nebo Time-of-Flight (ToF). 

Vytvořit stereokameru lze i pomocí dvou obyčejných kamer. Podrobný návod pro zájemce je k dispozici na příklad [zde](https://erget.wordpress.com/2014/02/01/calibrating-a-stereo-camera-with-opencv/).

## Typy systémů
<table>
    <tr>
        <th align="left" width="16%"><a href="https://en.wikipedia.org/wiki/Kinect">Microsoft Kinect</a></th>
        <th align="left" width="16%"><img src="images/kinect.png" width="50%"/></th>
        <th align="left" width="16%"><a href="https://en.wikipedia.org/wiki/Intel_RealSense">Intel RealSense</a></th>
        <th align="left" width="16%"><img src="images/realsense.png" width="50%"/></th>
        <th align="left" width="16%"><a href="https://en.wikipedia.org/wiki/Time-of-flight_camera">Basler ToF</a></th>
        <th align="left" width="16%"><img src="images/tof.png" width="50%"/></th>       
    </tr>
    <tr>
        <td align="left" colspan="2">Postaven na principu promítání laserového vzoru a měření jeho deformace infračervenou kamerou.</td>
        <td align="left" colspan="2">Využívá stejného principu jako Kinect, pouze má jiný vzor.</td>
        <td align="left" colspan="2">Využívá promítání světla z více LED zdrojů a měří čas jeho návratu (odrazu).</td>
    </tr>
    <tr>
        <td align="left" colspan="2">Je náchylný na měření venku. Denní světlo kazí obraz.</td>
        <td align="left" colspan="2">Měl by být vhodný i pro měření venku.</td>
        <td align="left" colspan="2">Je náchylný na měření venku. Denní světlo kazí obraz.</td>
    </tr>
    <tr>
        <td align="left" colspan="2">Více Kinectů vedle sebe si vzájemně kazí vzory.</td>
        <td align="left" colspan="2">Nejnovější a nejmenší senzor na trhu.</td>
        <td align="left" colspan="2">Průmyslová konstrukce pro běh 24/7.</td>
    </tr>
</table>

### Úkol
Cílem cvičení je využít hloubkovou mapu ze senzoru **Basler ToF** pro detekci a změření zkoumaného objektu. K tomu je zapotřebí být schopen počítat reálné rozměry objektů v **hloubkových mapách****.

Hloubkovou kameru budeme využívat k výpočtu objemu objektu. Jako vhodný objekt poslouží kostka s názvem **Cultilene**, která slouží jako podloží k pěstování rostlin a zeleniny. 

Její hlavní výhodou je pravidelný tvar a i trochu vylepšený vnitřek o vyfrézované části pro sazenice rostliny.

![](images/cultilene_both.png)

K výpočtu reálných rozměrů objektů v hloubkové mapě je zapotřebí znát typ senzoru, se kterým se pracuje a jeho zorný úhel jak v horizontálním tak ve vertikálním směru (tzv. FOV). Využívá se geometrie viz následující obrázek (FOV z obrázku **NENÍ** naše hledané). 

![](https://i.stack.imgur.com/Wf3bu.png)

### Import knihoven a konfigurace

In [ ]:
import os
import io

import cv2
import numpy as np
import matplotlib.pyplot as plt

from improutils import *

%matplotlib inline
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

### Pomocné funkce
- [normalize](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/preprocessing/preprocessing.py#L65)
- [funkce pro segmentaci](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/segmentation/segmentation.py)
- [negative](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/preprocessing/preprocessing.py#L50)
- [plot_images](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/visualisation/visualisation.py#L11)
- [find_contours](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/preprocessing/contours.py#L38)


#### 0) Zobrazte si obrázek hloubkové mapy kostky pro představu.

In [ ]:
viz_depth = ...('...') ###
plot_images(viz_depth)

#### 1) Najděte správné hodnoty FOV pro váš senzor a doplňte.
Specifikace kamery: https://machinevisionstore.com/catalog/details/1607

In [ ]:
FOV_HORIZONTAL_DEGREES = ... ###
FOV_VERTICAL_DEGREES = ... ###

def real_measure(w_px, h_px, depth, frame_width, frame_height):
    """
    Function for counting the real measures in meters out of pixel values in depth map images.
    Source: https://stackoverflow.com/a/45481222/1398955
    
    @param w_px : int
        Width in pixels.
    @param h_px : int
        Height in pixels.
    @param depth : float
        Depth value in meters.
    @param frame_width : int
        Frame width in pixels.
    @param frame_height : int
        Frame height in pixels.
    @return w_m : int
        Width in meters.
    @return h_m : int
        Height in meters.
    """
    fov_horizontal = FOV_HORIZONTAL_DEGREES * np.pi / 180.0
    fov_vertical = FOV_VERTICAL_DEGREES * np.pi / 180.0

    horizontal_scaling = 2 * np.tan(fov_horizontal / 2.0) / float(frame_width)
    vertical_scaling = 2 * np.tan(fov_vertical / 2.0) / float(frame_height)

    w_m = w_px * horizontal_scaling * depth
    h_m = h_px * vertical_scaling * depth
    
    return w_m, h_m

#### 2) Načtěte data hloubkové mapy a zobrazte.
Data z hloubkové kamery jsou datového typu float64, pro využití chytřejší segmentačních metod je třeba data převézt na typ uint8. Pro převedení dat do rozsahu [0, 255] je třeba použít normalizaci, OpenCV obsahuje vhodnou funkci.

Načtěte data hloubek pomocí metody [np.loadtxt](https://docs.scipy.org/doc/numpy/reference/generated/numpy.loadtxt.html). Následně obrázek normalizujte a segmentujte masku kostky.

In [ ]:
depths = ...('...') ### načtěte data hloubek
depths_normalized = ...(depths) ### normalizujte

im_seg = ...(depths_normalized) ### segmentujte kostku

plot_images(im_seg, depths_normalized)

#### 3) Získejte masku měřeného objektu.
Pro měření objektu je třeba získat konturu objektu. Zároveň je ale třeba, aby objekt neměl žádné díry. 

In [ ]:
### najděte konturu objektu a zároveň vytvořte jeho masku
cube_mask = ...
cube_contour = ...

plot_images(cube_mask)

#### 4) Změřte vzdálenost povrchu objektu.
S pomocí získané masky získejte hodnoty vzdáleností pixelů povrchu objektu z **hloubkové mapy**. S jejich pomocí zjistěte **správnou** hodnotu **vzdálenosti povrchu** objektu od senzoru.

Vodítka:
- OpenCV obsahuje vhodnou funkci pro logické sloučení dvou obrázků s využitím masky.
- Indexací nenulových hodnot lze také udělat crop.

**Pozor** na následující:
- Na povrchu objektu jsou i výřezy s rozdílnou hloubkou.
- Snímek nebyl pořízen v idealních podmínkách (rovina snímače nebyla rovnoběžná s rovinou objektu) a hodnoty na povrchu se tím od sebe trochu liší.

Zvolte vhodnou statistickou funkci pro získání rozumné hodnoty vzdálenosti povrchu (při zvolení správné funkce si numpy poradí i s nan hodnotami).

In [ ]:
cube_depths = ...(depths, depths, mask=cube_mask) ###
cube_depths = cube_depths[cube_depths != 0]

### najděte vhodnou funkci z numpy
top_depth = ...(cube_depths) ###
print('Vzdálenost povrchu objektu od senzoru je: ' + str(top_depth) + ' mm.')

#### 5) Změřte vzdálenost povrchu na kterém objekt stojí.
**Upravte** masku získanou v předešlém příkladu tak, abyste získali pouze hodnoty vzdáleností povrchu, na kterém objekt stojí.

Podobně jako minule **změřte vzdálenost**. Zamyslete se, které funkce z numpy jsou vhodné k zisku správné hodnoty a **vypište je s komentářem**. Jedna nestačí.

In [ ]:
bottom_mask = ...(cube_mask) ###

bottom_depths = ...(depths, depths, mask=bottom_mask) ###
bottom_depths = bottom_depths[bottom_depths != 0]

### najděte vhodnější funkci z numpy
# popis několika funkcí
# .
# .
#.
bottom_depth = ...(bottom_depths) ###
print('Vzdálenost povrchu, na kterém objekt stojí od senzoru je: ' + str(bottom_depth) + ' mm.')

#### 6) Změřte šířku a délku objektu.
K dispozici máme již všechny potřebné údaje pro změření reálných rozměrů objektu. 

Změřte velikost objektu nejdříve v **pixelech** a následně v **milimetrech**. Pro měření v pixelech využijte ideálně konturu objektu získanou v 3). Pro měření v milimetrech možná bude potřeba si znovu přečíst [zadání úkolu](#Úkol).

In [ ]:
### meření v pixelech
rect = ...(cube_contour) 
w_px, h_px = ...

### meření v milimetrech
real_width, real_height = ...

print(f'Změřená délka objektu je {real_height:.1f} mm a jeho šířka je {real_width:.1f} mm.')

#### 7) Výpočtěte výšku objektu a jeho přibližný objem.
Výšku i objem udávejte v cm, resp. cm$^3$.

In [ ]:
object_height = ... ###
volume_almost = ... ###

print(f'Výška objektu je {object_height:.1f} cm.')
print(f'Přibližný objem objektu je {volume_almost:.1f} cm^3.')

---

#### 8) Bonusová část - Výpočet korektního objemu
Nabízím až **2 body** aktivity za výpočet **korektního** objemu kostky. Je potřeba si uvědomit, že kostka má vyfrézované drážky pro sazenice rostlin a proto výše změřený objem není korektní.

Pomocí funkcí zpracování obrazu dodělejte úlohu do zdárného konce.